#### We need to install module future, not importing from \_\_future\_\_

In [ ]:
#from future.utils import PY3
import future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import pandas as pd
import numpy as np
import os
import pprint
from matplotlib import pyplot as plt

#### Load files

In [ ]:
# List dir
#mydir = '../_share/Losonczi/msa1215_1/'
mydir = '../_share/Losonczi/msa0316_1/'
os.listdir(mydir)

In [ ]:
# Load files
experiment_traits = pd.read_hdf(os.path.join(mydir,'experiment_traits.h5'),key='table')
raw_data = pd.read_hdf(os.path.join(mydir,'raw_data.h5'),key='table')
df_data = pd.read_hdf(os.path.join(mydir,'df_data.h5'),key='table')
transients_data = pd.read_hdf(os.path.join(mydir,'transients_data.h5'),key='table')
behavior_data = pd.read_hdf(os.path.join(mydir,'behavior_data.h5'),key='table')

In [ ]:
# Show headers
variables = ['experiment_traits', 'raw_data', 'df_data', 'transients_data', 'behavior_data']
for myname in variables:
    myvar = globals()[myname]
    print(myname, myvar.columns.values, myvar.shape)

# IS IT REALLY <font color="red">8 FPS</font>?
## Moreover, are we missing the first 1sec of almost all sessions?

In [ ]:
# Set event lengths
durations=np.array([0,10,20,15,5])
events=np.cumsum(durations)
FPS=8

#### Display data

In [ ]:
experiment_traits.head(10)

In [ ]:
et = experiment_traits.copy()
et['sum'] = 1
et.groupby(['learning_epoch','context','puffed','licking']).sum()

In [ ]:
# Look for an ID where there was conditional stimulus [sound paired with unconditional stimulus] (CS+), learning experience and unconditional stimulus (US)
allcond = experiment_traits[(experiment_traits.context=='CS+') & (experiment_traits.learning_epoch=='Post-Learning') & (experiment_traits.puffed==True)]
experiment_id = allcond.ix[0]['time']
allcond.head()

In [ ]:
# Information on licking
behavior_data.head(10)

In [ ]:
# Filtered calcium levels
df_data.head()

In [ ]:
# component count in a composite index, dataset size (index size x camera frames)
print(np.array(df_data.index.levels).shape, df_data.shape)
# index components are experiment IDs and neural unit IDs
ix0 = df_data.index.get_level_values(0).unique()
ix1 = df_data.index.get_level_values(1).unique()
print(ix0,ix0.shape,'\n',ix1,ix1.shape)
# Plot sample calcium levels
nsamples = 5
sel_df = raw_data.loc[experiment_id].ix[0:nsamples,:]
sel_time = (0, len(sel_df.columns))
sel_units = sel_df.index.values
sel_data = sel_df.values
%matplotlib inline
plt.plot(sel_data.T)
plt.title('Raw data')
plt.xlabel('Camera frame')
plt.ylabel('Raw Ca level')
plt.show()
data = df_data.loc[experiment_id].ix[0:5,:]
%matplotlib inline
plt.plot(data.T)
plt.title('Filtered data')
plt.xlabel('Camera frame')
plt.ylabel('Processed Ca level')
plt.show()

In [ ]:
# Information about spikes
transients_data.head(10)

In [ ]:
print(sel_units)
%matplotlib inline
# Show the same figure as above
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,8), sharex=True)
ax1.plot(data.T)
ax1.set_title('Filtered data')
ax1.set_xlabel('Camera frame')
ax1.set_ylabel('Processed Ca level')

# Plot transient maximum times for the same neural units
max_places = transients_data.loc[experiment_id]['max_frame']
for i in range(0,len(sel_units)):
    unit = sel_units[i]
    firing = max_places.loc[unit]
    ax2.plot(firing,i*np.ones_like(firing),'x',ms=10)
start_end = transients_data.loc[experiment_id][['start_frame', 'stop_frame']]
for i in range(0,len(sel_units)):
    unit = sel_units[i]
    firing = np.array(start_end.loc[unit])
    ax2.plot(firing.T,i*np.ones_like(firing.T),'k')
ax2.set_ylim((-1,len(sel_units)))
ax2.set_title('Transient peaks and durations')
ax2.set_xlabel('Camera frame')
ax2.set_ylabel('Unit ID')
plt.show()

In [ ]:
# Plot all neural units in this experiment
ixt = transients_data.loc[experiment_id].index.unique()
plt.figure(figsize=(16,10))
for i in range(0,len(ixt)):
    unit = ixt[i]
    firing = transients_data.loc[(experiment_id,unit),'max_frame']
    plt.plot(firing,i*np.ones_like(firing),'|',ms=5)
for i in range(0,len(events)):
    plt.axvline(x=events[i]*FPS, ymin=0.0, ymax = 1.0, linewidth=1, color='k')
plt.title('Transient peaks')
plt.xlabel('Camera frame')
plt.ylabel('Unit ID')
plt.show()

In [ ]:
# Plot all neural units in this experiment
ixt = transients_data.loc[experiment_id].index.unique()
plt.figure(figsize=(16,10))
for i in range(0,len(ixt)):
    unit = ixt[i]
    firing = np.array(transients_data.loc[(experiment_id,unit),['start_frame', 'stop_frame']])
    plt.plot(firing.T,i*np.ones_like(firing.T),c='k')
    firing = transients_data.loc[(experiment_id,unit),'max_frame']
    plt.plot(firing,i*np.ones_like(firing),'|',ms=5)
for i in range(0,len(events)):
    plt.axvline(x=events[i]*FPS, ymin=0.0, ymax = 1.0, linewidth=1, color='k')
plt.title('Transient peaks and durations')
plt.xlabel('Camera frame')
plt.ylabel('Unit ID')
plt.show()

In [ ]:
# The same in tabular format
transients_data.loc[experiment_id]

In [ ]:
import pprint, pickle

# Some undocumented info about the experiments
pkl_file = open('../_share/Losonczi/msa0316_1/frame_fluor.pkl', 'rb')

# Python 2.7
data1 = pickle.load(pkl_file)

# Python 3.5
#u = pickle._Unpickler(pkl_file)
#u.encoding = 'latin1'
#data1 = u.load()
    
#pprint.pprint(data1)

pprint.pprint(data1.keys())
pprint.pprint(data1.values()[0]) # same as data1['2016-04-01-23h43m20s']

In [ ]:
avail = df_data.shape[0]-df_data.isnull().sum()
plt.plot(avail)
plt.xlabel('Camera frame within experiment')
plt.ylabel('Available clusters')